In [4]:
import ee
import geemap
ee.Initialize()

In [5]:
landsat8Sr = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
roi = ee.Geometry.Point([-80.498, 37.254])

# Function to cloud mask from the pixel_qa band of Landsat 8 SR data.
def maskL8sr(image):
  # Bit 0 - Fill
  # Bit 1 - Dilated Cloud
  # Bit 2 - Cirrus
  # Bit 3 - Cloud
  # Bit 4 - Cloud Shadow
  qaMask = image.select('QA_PIXEL').bitwiseAnd(31).eq(0)
  saturationMask = image.select('QA_RADSAT').eq(0)

  # Apply the scaling factors to the appropriate bands.
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

  # Replace the original bands with the scaled ones and apply the masks.
  return image \
    .addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)



In [6]:


# Function to add NDVI, time, and constant variables to Landsat 8 imagery.
def addVariables(image):
  # Compute time in fractional years since the epoch.
  date = image.date()
  years = date.difference(ee.Date('1970-01-01'), 'year')
  # Return the image with the added bands.
  return image \
    .addBands(image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')) \
    .addBands(ee.Image(years).rename('t')).float() \
    .addBands(ee.Image.constant(1)) \

# Remove clouds, add variables and filter to the area of interest.
filteredLandsat = landsat8Sr \
  .filterBounds(roi) \
  .filterDate('2013', '2022') \
  .map(maskL8sr) \
  .map(addVariables)

In [8]:
# List of the independent variable names.
independents = ee.List(['constant', 't'])
# Name of the dependent variable.
dependent = ee.String('NDVI')
# Compute a linear trend. This will have two bands: 'residuals' and
# a 2x1 band called 'coefficients' (columns are for dependent variables).
trend = filteredLandsat.select(independents.add(dependent)) \
  .reduce(ee.Reducer.linearRegression(independents.length(), 1))

Map = geemap.Map()
Map.addLayer(trend, {}, 'Trend Array Image')
Map.centerObject(roi, 10)

Map

Map(center=[37.254, -80.498], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [11]:

# Flatten the coefficients into a 2-band image.

bandNames = [['constant', 't'], ['NDVI']]
coefficients = trend.select('coefficients') #\
  # .arrayProject([0]) \
  # .arrayFlatten([independents])

coeff_mat = coefficients.arrayFlatten(bandNames) #.getInfo()

In [12]:
coeff_mat.getInfo()

{'type': 'Image',
 'bands': [{'id': 'constant_NDVI',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 't_NDVI',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}